# Implementation of State Space Model

__Source:__  
https://medium.com/@malintha/state-estimation-with-kalman-filter-8e44eed4ad2f  
https://www.youtube.com/watch?v=IFeCIbljreY&ab_channel=VisuallyExplained
http://www-control.eng.cam.ac.uk/gv/p6/index.html  

__Note:__ 
* What it means to do matrix conjugation ($ABA^{-1}$)? This pattern occurs when there is an orthonormal change of basis.  
    Link: https://stats.stackexchange.com/questions/331899/whats-the-intuition-for-aba-in-linear-algebra  
* Change of basis is often confused with linear transformation.  
    Link: http://www.boris-belousov.net/2016/05/31/change-of-basis/  

In [1]:
import numpy as np

In [ ]:
# EXAMPLE 1: Motion System
class StateSpaceModel:
    def __init__(self):
        # Assume we are interested in a motion system measured by distance and speed, and subjected by acceleration as user input:
        #   * distance: s_{t+1} = s_t  + (ds_t * dt) + (1/2 * a * dt^2)
        #   * speed: ds_{t+1} = ds_t +  (a * dt)
        #
        # Then, we can combine the two states for the system using linear algebra:
        #   | s_{t+1}  |  =  | 1  dt || s_t  | + | (dt^2)/2 | a
        #   | ds_{t+1} |     | 0   1 || ds_t |   |    dt    |
        #
        # This is called a state space model with generic form below:
        #   * X_{t+1} = A*X_t + B*U_t + w_t 
        #   * Y_{t+1} = C*X_{t+1} + G*ff_{t+1} +v_{t+1}
        #
        # In this case, we have:
        #   * state parameter: A   = [[1, dt], [0, 1]]  
        #   * state matrix:    X_t = < s_t | ds_t >     
        #   * input parameter: B   = [[1, dt], [0, 1]]  
        #   * input matrix:    U_t = a
        
        self.t = 0
        # state equation
        self.X_pred = None
        self.A = np.matrix([[1, 0], 
                            [0, 1]])          # `A` applies the effect of current system state parameter on the next state vector
        self.X = np.array([0, 0])             # `X` is the current system state parameters
        self.B = np.array([1/2, 1])           # `B` applies the effect of current input parameters on the next state vector
        self.U = np.array([0, 0])             # `U` is the current input parameters
        self.w = np.array([0, 0])             # `w` contains noise for each term in state vector. Noise is assumed to be drawn from 
                                              #     a zero mean multivariate Gaussian distribution, with covariane given by Q_t matrix. 
        
        # measurement equation
        self.Y_pred = None
        self.Y = np.array([0, 0])             # `Y` is the vector of measurement
        self.C = np.matrix([[1, 0], 
                            [0, 1]])          # `C` is the transformation matrix that maps the state parameters to measurement domain
        self.G = np.matrix([[1, 0], 
                            [0, 1]])          # `G` applies the effect of incoming variations on the measurement 
        self.ff = np.matrix([[0, 0], 
                             [0, 0]])         # `ff` is the vector of incoming variations
        self.v = np.array([0, 0])             # `v` contains noise for each term in measurement vector. Noise is assumed to be drawn from 
                                              #     a zero mean multivariate Gaussian distribution, with covariane given by R_t matrix.
                                              
        # Kalman Filter and state covariance estimation
        self._K = np.array([0, 0])            # `W` is called Kalman Gain in state covariance estimation
        self._P = np.array([0, 0])            # `P` is called State covariance
        self._P_pred = np.array([0, 0])       # `P_pred` is called State prediction covariance, represent the systemic errors (e.g. bias) in estimated states
        self._S_pred = np.array([0, 0])       # `S_pred` is called measurement prediction covariance, represent the systemic errors (e.g. bias) in measurement
        self._Q = np.matrix([[100, 0],        # `Q` is covariance matrix of state noise, that describe the noise distribution to be sampled from
                             [0, 100]])       #     <top-left to bottom-right> Diagonal elements represent the uncertainty in the respective states
                                              #     The other elements represent the correlation between noises in the different pairs of states
        self._R = np.matrix([[1, 0],          # `R` is covariance matrix of measurement noise, that describe the noise distribution to be sampled from
                             [0, 1]])         #     <top-left to bottom-right> Diagonal elements represent the uncertainty in the respective measurements
                                              #     The other elements represent the correlation between noises in the different pairs of measurements
        
    
    # Step 1: Prediction Step
    def predict_state(self):
        # update at given time, before measurement can be made (assumes no noise)
        self.X_pred = (self.A @ self.X) + (self.B @ self.U)
        # estimate errors in state (bias and Q-matrix noise)
        self._P_pred = (self.A @ self._P @ self.A.T) + self._Q
    
    def predict_measurement(self):
        # predict measurement based on the predicted state
        self.Y_pred = (self.C @ self.X_pred) + (self.G @ self.ff)
        # estimate errors in measurement (bias and R-matrix noise)
        self._S_pred = (self.C @ self._P_pred @ self.C.T) + self._R
        
        
    # Step 2: Update Step
    def update_kalman_gain(self):
        # update the Kalman Gain "weight" of the measurement delta, this is a multiplier to measurement noise (w).
        ## self._P_pred indicates how much you trust your prediction. Lower is more trust.
        ## self._S_pred indicates how much you trust your measurement. Lower is more trust.
        ## K_i is always between 0 and 1.
        ## If K_i = 0, this means _P is zero, meaning you have complete trust in your prediction.
        ## If K_i = 1, this means _S is zero, meaning you have complete trust in your measurement.
        self._K = self._P_pred @ self.C.T @ np.linalg.inv(self._S_pred)
        # update state noise for current timestep
        # self._P = self._P_pred - self._K @ self._S_pred @ self._K.T  
        self._P = (np.identity(self._S_pred.shape[0]) - self._K @ self.C) @ self._S_pred # ?? wikipedia version
        
    def update_measurement_residual(self, Y):
        # record that measurement is made 
        self.Y = Y
        # calculate delta from predicted measurement
        self.w = self.Y - self.Y_pred
        # correct the state with measurement delta, attenuated using Kalman Gain (_K) to account for noise.
        self.X = self.X_pred + (self._K @ self.w)
        

IndentationError: expected an indented block after function definition on line 53 (497027172.py, line 55)

In [12]:
# EXAMPLE 2: Semiconductor processing
class StateSpaceModel:
    def __init__(self):
        # Assume we have a single-input, single-output deposition system.
        # Deposition rate is 2 nm per second with target of 10 nm, and baseline deposition time of 5nm.
        # We can keep states and input separate to prevent interactions as shown below.
        
        self.t = 0
        self.horizon = 5
        self.Y_hist = [np.ndarray(0),] * self.horizon
        
        # state equation
        self.A = np.matrix([[0, 0], [0, 1]])
        self.X = np.array([0, 0])
        self.B = np.matrix([[2, 0], [0, 0]])
        self.U = np.array([5, 0])
        self.w = np.array([0, 0])
        
        # measurement equation
        self.Y = np.array([100, 0])
        self.F = np.matrix([[1, 0], [0, 1]])
        self.G = np.matrix([[1, 0], [0, 1]])
        self.ff = np.array([0, 0])
        self.v = np.array([0, 0])
        
        # kalman filter
        self.Q = np.matrix([[1, 0], [0, 1]])
        self.R = np.matrix([[1000, 0], [0, 1000]])
    
    def update_measurement(self, Y):
        self.Y_hist.pop(0)
        self.Y_hist.append(Y)
        
        
        